In [2]:
try:
    # one of the ways to check if our notebook is running inside google colab
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    !git clone https://github.com/gmum/ml2025-26.git
    import sys
    sys.path.append('/content/ml2025-26/lab')

Cloning into 'ml2025-26'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 144 (delta 56), reused 127 (delta 41), pack-reused 0 (from 0)
Receiving objects: 100% (144/144), 22.08 MiB | 22.19 MiB/s, done.
Resolving deltas: 100% (56/56), done.


### Importy i Utilsy  (odpalić i schować )

In [3]:
import os
import unicodedata
import string
from typing import Tuple, List

import torch
import numpy as np
from torch.nn.functional import cross_entropy
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import f1_score

all_letters = string.ascii_letters
n_letters = len(all_letters)


class ListDataset(Dataset):

    def __init__(self, data, targets):

        self.data = data
        self.targets = targets

    def __getitem__(self, ind):

        return self.data[ind], self.targets[ind]

    def __len__(self):
        return len(self.targets)


def unicode_to__ascii(s: str) -> str:
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'
                                                                 and c in all_letters)


def read_lines(filename: str) -> List[str]:
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicode_to__ascii(line) for line in lines]


def letter_to_index(letter: str) -> int:
    return all_letters.find(letter)


def line_to_tensor(line: str) -> torch.Tensor:
    tensor = torch.zeros(len(line), n_letters)
    for i, letter in enumerate(line):
        tensor[i][letter_to_index(letter)] = 1
    return tensor

## Dane sekwencyjne

Modele, którymi zajmowaliśmy się wcześniej zakładały konkretny kształt danych. Dla przykładu klasyczna sieć neuronowa fully-connected dla MNISTa zakładała, że na wejściu dostanie wektory rozmiaru 784 - dla wektorów o innej wymiarowości i innych obiektów model zwyczajnie nie będzie działać.

Takie założenie bywa szczególnie niewygodne przy pracy z niektórymi typami danych, takimi jak:
* językiem naturalny (słowa czy zdania mają zadanej z góry liczby znaków)
* szeregi czasowe (dane giełdowe ciągną się właściwie w nieskończoność)
* dźwięk (nagrania mogą być krótsze lub dłuższe).

Do rozwiązania tego problemu służą rekuencyjne sieci neuronowe (*recurrent neural networks, RNNs*), które zapamiętują swój stan z poprzedniej iteracji.

### Ładowanie danych
Poniższe dwie komórki ściągają dataset nazwisk z 18 różnych narodowości. Każda litera w danym nazwisku jest zamieniana na jej indeks z alfabetu w postaci kodowania "one-hot". Inaczej mówiąc, każde nazwisko jest binarną macierzą rozmiaru `n_letters` $\times$ `len(name)`.

Dodatkowo, ponieważ ten dataset jest mocno niezbalansowany, użyjemy specjalnego samplera do losowania przykładów treningowych, tak aby do uczenia sieć widziała tyle samo przykładów z każdej klasy.

Ponieważ nazwiska mogą mieć różne długości będziemy rozważać `batch_size = 1` w tym notebooku (choć implementacje modeli powinny działać dla dowolnych wartości `batch_size`!)

In [9]:
!wget https://download.pytorch.org/tutorial/data.zip
!unzip data.zip

--2025-12-11 14:13:27--  https://download.pytorch.org/tutorial/data.zip
Resolving download.pytorch.org (download.pytorch.org)... 143.204.160.84, 143.204.160.77, 143.204.160.115, ...
Connecting to download.pytorch.org (download.pytorch.org)|143.204.160.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2882130 (2.7M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   2.75M  --.-KB/s    in 0.05s   

2025-12-11 14:13:27 (54.3 MB/s) - ‘data.zip’ saved [2882130/2882130]

Archive:  data.zip
   creating: data/
  inflating: data/eng-fra.txt        
   creating: data/names/
  inflating: data/names/Arabic.txt   
  inflating: data/names/Chinese.txt  
  inflating: data/names/Czech.txt    
  inflating: data/names/Dutch.txt    
  inflating: data/names/English.txt  
  inflating: data/names/French.txt   
  inflating: data/names/German.txt   
  inflating: data/names/Greek.txt    
  inflating: data/names/Irish.txt    
  inflating: data/nam

In [10]:
# NOTE: you can change the seed or remove it completely if you like
torch.manual_seed(1337)

data_dir = 'data/names'

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

data = []
targets = []
label_to_idx = {}

# read each natonality file and process data
for label, file_name in enumerate(os.listdir(data_dir)):

    label_to_idx[label] = file_name.split('.')[0].lower()

    names = read_lines(os.path.join(data_dir, file_name))
    data += [line_to_tensor(name) for name in names]
    targets += len(names) * [label]

# split into train and test indices
test_frac = 0.1
n_test = int(test_frac * len(targets))
test_ind = np.random.choice(len(targets), size=n_test, replace=False)
train_ind = np.setdiff1d(np.arange(len(targets)), test_ind)

targets = torch.tensor(targets)
train_targets = targets[train_ind]

# calculate weights for BalancedSampler
uni, counts = np.unique(train_targets, return_counts=True)
weight_per_class = len(targets) / counts
weight = [weight_per_class[c] for c in train_targets]
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights=weight, num_samples=len(weight))

train_dataset = ListDataset(data=[x for i, x in enumerate(data) if i in train_ind], targets=train_targets)
train_loader = DataLoader(train_dataset, shuffle=False, batch_size=1, sampler=sampler)

test_dataset = ListDataset(data=[x for i, x in enumerate(data) if i in test_ind], targets=targets[test_ind])
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=1)

In [ ]:
# check out the content of the dataset
for i, (x, y) in enumerate(train_loader):
    break

print("x.shape:", x.shape)
print("name: ", end="")
for letter_onehot in x[0]:
    print(all_letters[torch.argmax(letter_onehot)], end="")

print("\ny:", label_to_idx[y.item()])

x.shape: torch.Size([1, 4, 52])
name: Ruda
y: czech


## Zadanie 1. (2 pkt.)

Zaimplementuj "zwykłą" sieć rekurencyjną.
![rnn](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)

Przyjmijmy, że stan ukryty ma wymiarowość $H$. Wtedy komórka powinna być warstwą liniową o postaci: $\tanh(W^T [x_t, h_{t-1}] + b)$, gdzie $x_t \in \mathbb{R}^{D}$ to wejście w kroku $t$, $h_{t-1} \in \mathbb{R}^{H}$ to stan ukryty z poprzedniego kroku a $W \in \mathbb{R}^{(H + D) \times H}$ i $b \in \mathbb{R}^H$ to parametry naszego modelu.

* W klasie `RNN` należy zainicjalizować potrzebne wagi oraz zaimplementować główną logikę dla pojedynczej chwili czasowej $x_t$
* Wyjście z sieci może mieć dowolny rozmiar, potrzebna jest również warstwa przekształacjąca $H$-wymiarowy stan ukryty na wyjście (o takiej wymiarowości ile mamy klas w naszym problemie).
* W pętli uczenia należy dodać odpowiednie wywołanie sieci. HINT: pamiętać o iterowaniu po wymiarze "czasowym".


In [ ]:
class RNN(torch.nn.Module):

    def __init__(self,
                 input_size: int,
                 hidden_size: int,
                 output_size: int):
        """
        :param input_size: int
            Dimensionality of the input vector
        :param hidden_size: int
            Dimensionality of the hidden space
        :param output_size: int
            Desired dimensionality of the output vector
        """
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.input_to_hidden = torch.nn.Linear(input_size + hidden_size, hidden_size)

        self.hidden_to_output = torch.nn.Linear(hidden_size, output_size)

    # for the sake of simplicity a single forward will process only a single timestamp
    def forward(self,
                input: torch.tensor,
                hidden: torch.tensor) -> Tuple[torch.tensor, torch.tensor]:
        """
        :param input: torch.tensor
            Input tensor for a single observation at timestep t
            shape [batch_size, input_size]
        :param hidden: torch.tensor
            Representation of the memory of the RNN from previous timestep
            shape [batch_size, hidden_size]
        """

        combined = torch.cat([input, hidden], dim=1)
        hidden = self.input_to_hidden(combined)
        hidden = torch.tanh(hidden)
        output =  self.hidden_to_output(hidden)
        return output, hidden

    def init_hidden(self, batch_size: int) -> torch.Tensor:
        """
        Returns initial value for the hidden state
        """
        return torch.zeros(batch_size, self.hidden_size, requires_grad=True).cuda()

### Pętla uczenia

In [ ]:
n_class = len(label_to_idx)

# initialize network and optimizer
rnn = RNN(n_letters, 256, n_class).cuda()
optimizer = torch.optim.SGD(rnn.parameters(), lr=0.01)

# we will train for only a single epoch
epochs = 1


# main loop
for epoch in range(epochs):

    loss_buffer = []

    for i, (x, y) in enumerate(train_loader):

        x = x.cuda()
        y = y.cuda()

        optimizer.zero_grad()
        # get initial hidden state
        hidden = rnn.init_hidden(x.shape[0])

        # get output for the sample, remember that we treat it as a sequence
        # so you need to iterate over the 2nd, time dimension

        seq_len = x.shape[1]
        for n in range(seq_len):
            output, hidden = rnn(x[:, n, :], hidden)

        loss = cross_entropy(output, y)
        loss.backward()
        optimizer.step()

        loss_buffer.append(loss.item())

        if i % 1000 == 1:
            print(f"Epoch: {epoch} Progress: {100 * i/len(train_loader):2.0f}% Loss: {np.mean(loss_buffer):.3f}")
            loss_buffer = []


# evaluate on the test set
with torch.no_grad():
    ps = []
    ys = []
    correct = 0
    for i, (x, y) in enumerate(test_loader):
        x = x.cuda()
        ys.append(y.numpy())

        hidden = rnn.init_hidden(x.shape[0])
        seq_len = x.shape[1]

        for n in range(seq_len):
            output, hidden = rnn(x[:, n, :], hidden)
        pred = output.argmax(dim=1)
        ps.append(pred.cpu().numpy())

    ps = np.concatenate(ps, axis=0)
    ys = np.concatenate(ys, axis=0)
    f1 = f1_score(ys, ps, average='weighted')

    print(f"Final F1 score: {f1:.2f}")
    assert f1 > 0.15, "You should get over 0.15 f1 score, try changing some hyperparams!"

Epoch: 0 Progress:  0% Loss: 2.901
Epoch: 0 Progress:  6% Loss: 2.861
Epoch: 0 Progress: 11% Loss: 2.780
Epoch: 0 Progress: 17% Loss: 2.496
Epoch: 0 Progress: 22% Loss: 2.221
Epoch: 0 Progress: 28% Loss: 2.089
Epoch: 0 Progress: 33% Loss: 1.970
Epoch: 0 Progress: 39% Loss: 1.930
Epoch: 0 Progress: 44% Loss: 1.865
Epoch: 0 Progress: 50% Loss: 1.828
Epoch: 0 Progress: 55% Loss: 1.829
Epoch: 0 Progress: 61% Loss: 1.734
Epoch: 0 Progress: 66% Loss: 1.634
Epoch: 0 Progress: 72% Loss: 1.701
Epoch: 0 Progress: 77% Loss: 1.636
Epoch: 0 Progress: 83% Loss: 1.582
Epoch: 0 Progress: 89% Loss: 1.629
Epoch: 0 Progress: 94% Loss: 1.627
Epoch: 0 Progress: 100% Loss: 1.587
Final F1 score: 0.19


## Zadanie 2. (0.5 pkt.)
Zaimplementuj funkcje `predict`, która przyjmuje nazwisko w postaci stringa oraz model RNN i wypisuje 3 najlepsze predykcje narodowości dla tego nazwiska razem z ich logitami.

**Hint**: Przyda się tutaj jedna z funkcji z pierwszej komórki notebooka.

In [ ]:
def predict(name: str, rnn: RNN):
    """Prints the name and model's top 3 predictions with scores"""
    name_vec = line_to_tensor(name)
    name_vec = name_vec.unsqueeze(0).cuda()
    n_chars = name_vec.shape[1]
    hidden = rnn.init_hidden(name_vec.shape[0])

    for n in range(n_chars):
        output, hidden = rnn(name_vec[:, n, :], hidden)
    pred = torch.topk(output, 3).indices.cpu()[0]

    for idx in pred:
        print(label_to_idx[idx.item()])

In [ ]:
some_names = ["Satoshi", "Jackson", "Schmidhuber", "Hinton", "Kowalski"]

for name in some_names:
    print(name)
    predict(name, rnn)
    print("=" * 10)

Satoshi
italian
japanese
polish
Jackson
scottish
english
korean
Schmidhuber
dutch
german
czech
Hinton
scottish
english
russian
Kowalski
polish
russian
czech


## Zadanie 3 (4 pkt.)
Ostatnim zadaniem jest implementacji komórki i sieci LSTM.

![lstm](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)

* W klasie `LSTMCell` ma znaleźć się główna logika LSTMa, czyli wszystkie wagi do stanów `hidden` i `cell` jak i bramek kontrolujących te stany.
* W klasie `LSTM` powinno znaleźć się wywołanie komórki LSTM, HINT: poprzednio było w pętli uczenia, teraz przenisiemy to do klasy modelu.
* W pętli uczenia należy uzupełnić brakujące wywołania do uczenia i ewaluacji modelu.

Zdecydowanie polecam [materiały Chrisa Olaha](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) do zarówno zrozumienia jak i ściągi do wzorów.

Zadaniem jest osiągnięcie wartości `f1_score` lepszej niż na sieci RNN, przy prawidłowej implementacji nie powinno być z tym problemów używając podanych hiperparametrów. Dozwolona jest oczywiście zmiana `random seed`.

#### Komórka LSTM

In [15]:
class LSTMCell(torch.nn.Module):

    def __init__(self,
                 input_size: int,
                 hidden_size: int):
        """
        :param input_size: int
            Dimensionality of the input vector
        :param hidden_size: int
            Dimensionality of the hidden space
        """

        super(LSTMCell, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.forget_gate = torch.nn.Linear(input_size + hidden_size, hidden_size)
        self.input_gate = torch.nn.Linear(input_size + hidden_size, hidden_size)
        self.tanh_gate = torch.nn.Linear(input_size + hidden_size, hidden_size)
        self.output_gate = torch.nn.Linear(input_size + hidden_size, hidden_size)

    def forward(self,
                input: torch.tensor, #bs, seq_len, emb_size
                states: Tuple[torch.tensor, torch.tensor]) -> Tuple[torch.tensor, torch.tensor]:

        hidden, cell = states
        forget_gate_out = torch.sigmoid(self.forget_gate(torch.cat([input, hidden], dim=1)))
        input_gate_out = torch.sigmoid(self.input_gate(torch.cat([input, hidden], dim=1)))
        cell_tilde = torch.tanh(self.tanh_gate(torch.cat([input, hidden], dim=1)))
        cell = forget_gate_out * cell + input_gate_out * cell_tilde
        out_gate_out = torch.sigmoid(self.output_gate(torch.cat([input, hidden], dim=1)))
        hidden = out_gate_out * torch.tanh(cell)

        return hidden, cell

### Klasa modelu LSTM

In [21]:
class LSTM(torch.nn.Module):

    def __init__(self,
                 input_size: int,
                 hidden_size: int):
        """
        :param input_size: int
            Dimensionality of the input vector
        :param hidden_size: int
            Dimensionality of the hidden space
        """

        super(LSTM, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size

        self.cell = LSTMCell(input_size=input_size, hidden_size=hidden_size)

    def forward(self,
                input: torch.tensor) -> Tuple[torch.tensor, torch.tensor]:
        """
        :param input: torch.tensor
            A sequence of observations of shape [batch_size, seq_len, input_size]
        Returns Tuple of two torch.tensors, both of shape [seq_len, batch_size, hidden_size]
        """

        batch_size = input.shape[0]

        hidden, cell = self.init_hidden_cell(batch_size)

        time_steps = input.shape[1]
        for t in range(time_steps):
            hidden, cell = self.cell(input[:, t, :], (hidden, cell))

        return hidden, cell

    def init_hidden_cell(self, batch_size):
        """
        Returns initial value for the hidden and cell states
        """
        return (torch.zeros(batch_size, self.hidden_size, requires_grad=True).cuda(),
                torch.zeros(batch_size, self.hidden_size, requires_grad=True).cuda())

### Pętla uczenia

In [24]:
from itertools import chain

# torch.manual_seed(1337)

# build data loaders
train_loader = DataLoader(train_dataset, batch_size=1, sampler=sampler)
test_loader = DataLoader(test_dataset, batch_size=1)

# initialize the lstm with an additional classifier layer at the top
lstm = LSTM(input_size=len(all_letters), hidden_size=128).cuda()
clf = torch.nn.Linear(in_features=128, out_features=len(label_to_idx)).cuda()

# initialize a optimizer
params = chain(lstm.parameters(), clf.parameters())
optimizer = torch.optim.Adam(params, lr=0.01)

# we will train for only a single epoch
epoch = 1

# main loop
for epoch in range(epoch):

    loss_buffer = []

    for i, (x, y) in enumerate(train_loader):

        x = x.cuda()
        y = y.cuda()

        optimizer.zero_grad()

        # don't forget about the classifier!

        hidden, _ = lstm(x)
        output = clf(hidden)

        # calculate the loss
        loss = cross_entropy(output, y)
        loss.backward()
        optimizer.step()

        loss_buffer.append(loss.item())

        if i % 1000 == 1:
            print(f"Epoch: {epoch} Progress: {100 * i/len(train_loader):2.0f}% Loss: {np.mean(loss_buffer):.3f}")
            loss_buffer = []

# evaluate on the test set
with torch.no_grad():
    ps = []
    ys = []
    for i, (x, y) in enumerate(test_loader):

        x = x.cuda()
        ys.append(y.numpy())

        hidden, _ = lstm(x)
        output = clf(hidden)

        pred = output.argmax(dim=1)
        ps.append(pred.cpu().numpy())

    ps = np.concatenate(ps, axis=0)
    ys = np.concatenate(ys, axis=0)
    f1 = f1_score(ys, ps, average='weighted')

    print(f"Final F1 score: {f1:.2f}")
    assert f1 > 0.18, "You should get over 0.18 f1 score, try changing some hiperparams!"

Epoch: 0 Progress:  0% Loss: 2.916
Epoch: 0 Progress:  6% Loss: 2.516
Epoch: 0 Progress: 11% Loss: 1.884
Epoch: 0 Progress: 17% Loss: 1.714
Epoch: 0 Progress: 22% Loss: 1.568
Epoch: 0 Progress: 28% Loss: 1.399
Epoch: 0 Progress: 33% Loss: 1.363
Epoch: 0 Progress: 39% Loss: 1.301
Epoch: 0 Progress: 44% Loss: 1.274
Epoch: 0 Progress: 50% Loss: 1.192
Epoch: 0 Progress: 55% Loss: 1.145
Epoch: 0 Progress: 61% Loss: 1.145
Epoch: 0 Progress: 66% Loss: 0.996
Epoch: 0 Progress: 72% Loss: 1.069
Epoch: 0 Progress: 77% Loss: 1.028
Epoch: 0 Progress: 83% Loss: 1.054
Epoch: 0 Progress: 89% Loss: 1.010
Epoch: 0 Progress: 94% Loss: 1.006
Epoch: 0 Progress: 100% Loss: 0.946
Final F1 score: 0.21


## Zadanie 4. (0.5 pkt.)
Zaimplementuj analogiczną do funkcji `predict` z zadania 2 dla modelu `lstm+clf`.


In [42]:
def predict_lstm(name: str, lstm: LSTM, clf: torch.nn.Module):
    """Prints the name and model's top 3 predictions with scores"""
    name_vec = line_to_tensor(name)
    name_vec = name_vec.unsqueeze(0).cuda()

    # The LSTM forward method already handles iterating over the sequence.
    # So we just pass the entire name_vec to it to get the final hidden state.
    hidden, _ = lstm(name_vec)
    output = clf(hidden)
    pred = torch.topk(output, 3).indices.cpu()[0]

    for idx in pred:
        print(label_to_idx[idx.item()])

In [43]:
# test your lstm predictor
some_names = ["Satoshi", "Jackson", "Schmidhuber", "Hinton", "Kowalski"]

for name in some_names:
    print(name)
    predict_lstm(name, lstm, clf)
    print("=" * 10)

Satoshi
japanese
italian
arabic
Jackson
scottish
russian
english
Schmidhuber
german
dutch
english
Hinton
english
scottish
german
Kowalski
polish
czech
russian
